In [1]:
#!pip install requests
#!pip install prettytable
!pip install pandas
!pip install pymysql
#!pip install google.colab

  Using cached PyMySQL-0.9.3-py2.py3-none-any.whl (47 kB)


In [20]:
!python --version

import pymysql
import pandas as pd

Python 3.7.6


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
import oshttp://localhost:8888/notebooks/Documents/Learnings/MSCA/Data%20Engineering/03%20-%20Assignments/Assignment-99/bls_api.ipynb#
arr = os.listdir()
print(arr)

In [21]:
#extract_df = pd.read_csv('Unemployment Series ID Codes - Final Codes.csv')
extract_df = pd.read_csv('inflationCode.csv')
#extract_df

In [ ]:
#extract_df.columns = ['code', 'sexes','races','age','education','marital_status','bls_type']

In [22]:
#extract_df[['Code']].to_numpy()
extract_df 

,Code
0,CUUR0000SA0
1,CUSR0000SA0


In [23]:
from itertools import chain 

code_list = extract_df[['Code']].values.tolist()
li = list(chain.from_iterable(code_list))
#li

In [24]:
li

['CUUR0000SA0', 'CUSR0000SA0']

In [ ]:
#@title Default title text
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": li[26:41] ,"startyear":"2005", "endyear":"2013"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=60256a471f4a427c813300a92445943c&catalog=false&startyear=2010&endyear=2021', data=data, headers=headers)
json_data = json.loads(p.text)

try:
      df = pd.DataFrame()
      for series in json_data['Results']['series']:
          df_initial = pd.DataFrame(series)
          series_col = df_initial['seriesID'][0]
          for i in range(0, len(df_initial) - 1):
              df_row = pd.DataFrame(df_initial['data'][i])
              df_row['seriesID'] = series_col
              if 'code' not in str(df_row['footnotes']): 
                  df_row['footnotes'] = ''
              else:
                  df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
              df = df.append(df_row, ignore_index=True)
      #return df
except:
      json_data['status'] == 'REQUEST_NOT_PROCESSED'
      print('BLS API has given the following Response:', json_data['status'])
      print('Reason:', json_data['message'])


In [25]:
import pandas as pd
import json
import requests

def get_bls_data(series, start, end):
    headers = {'Content-Type': 'application/json'}
    data = json.dumps({"seriesid": series,"startyear":"%d" % (start), "endyear":"%d" % (end)})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=60256a471f4a427c813300a92445943c&catalog=false&startyear=2010&endyear=2021', data=data, headers=headers)
    json_data = json.loads(p.text)
    try:
        df = pd.DataFrame()
        for series in json_data['Results']['series']:
            df_initial = pd.DataFrame(series)
            series_col = df_initial['seriesID'][0]
            for i in range(0, len(df_initial) - 1):
                df_row = pd.DataFrame(df_initial['data'][i])
                df_row['seriesID'] = series_col
                if 'code' not in str(df_row['footnotes']): 
                    df_row['footnotes'] = ''
                else:
                    df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
                df = df.append(df_row, ignore_index=True)
        return df
    except:
        json_data['status'] == 'REQUEST_NOT_PROCESSED'
        print('BLS API has given the following Response:', json_data['status'])
        print('Reason:', json_data['message'])
        return df



#writer = pd.ExcelWriter('bls.xlsx', engine='xlsxwriter', options={'strings_to_numbers': True})
#df.to_excel(writer, sheet_name='Sheet1', index=False)
#writer.save()

In [26]:
#Inflation
start = 2015
end = 2020
series = li
df1 = get_bls_data(series=series, start=start, end=end)
try:
  df1.count()
except:
  print("count failed: ",json_data['message'])

In [37]:
#df_final  = df1
df_final = df_final.fillna("")

In [ ]:
start = 2014
end = 2020
series = li[0:25]
df1 = get_bls_data(series=series, start=start, end=end)
try:
  df1.count()
except:
  print("count failed: ",json_data['message'])

In [ ]:
series1 = li[26:41]
df2 = get_bls_data(series=series1, start=start, end=end)
df2.count()

In [ ]:
start = 2005
end = 2013
series = li[0:25]
df3 = get_bls_data(series=series, start=start, end=end)
series1 = li[26:41]
df4 = get_bls_data(series=series1, start=start, end=end)
#df3.count()
df4.count()

In [ ]:
#li[26:41]

df_final = df1.append(df2, ignore_index = True)
df_final = df_final.append(df3, ignore_index = True)
df_final = df_final.append(df4, ignore_index = True)

#df_final.head()


In [29]:
df_final.head()
df_final.to_csv('staging_data_inflation_2015_2020_20200603.csv')

In [30]:
# Open database connection
db = pymysql.connect("35.202.249.234","root","rootroot","DEPA_final" )

# prepare a cursor object using cursor() method
cursor = db.cursor()

# execute SQL query using execute() method.
cursor.execute("SELECT VERSION()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()

print ("Database version : %s " % data)

# disconnect from server
#db.close()

Database version : 5.7.25-google-log 


In [ ]:
# Prepare SQL query.
sql = "CREATE TABLE t_bls_Unemployed_codes ( \
  stg_id SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT, \
  code VARCHAR(45), \
  sexes VARCHAR(4),\
  races VARCHAR(10), \
  age VARCHAR(10), \
  education VARCHAR(255), \
  marital_status varchar(45), \
  bls_type varchar(45), \
  ts TIMESTAMP DEFAULT CURRENT_TIMESTAMP, \
  PRIMARY KEY  (stg_id) \
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;"


try:
   # Execute the SQL command
    a = cursor.execute(sql)
    
   # Fetch all the rows in a list of lists.
   #rows = cursor.fetchall()
except:
    print ("Error: unable to fetch data");

db.commit()

#del df['footnotes']
# creating column list for insertion
cols = "`,`".join([str(i) for i in extract_df.columns.tolist()])


cols

 df.columns = ['a', 'b']

In [ ]:
# Prepare SQL query.
sql = "CREATE TABLE t_bls_cpi ( \
  stg_id SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT, \
  year VARCHAR(45), \
  period VARCHAR(45),\
  periodName VARCHAR(45), \
  latest VARCHAR(10), \
  value DOUBLE, \
  footnotes varchar(45), \
  seriesID varchar(45), \
  ts TIMESTAMP DEFAULT CURRENT_TIMESTAMP, \
  PRIMARY KEY  (stg_id) \
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;"


try:
   # Execute the SQL command
    a = cursor.execute(sql)
    
   # Fetch all the rows in a list of lists.
   #rows = cursor.fetchall()
except:
    print ("Error: unable to fetch data");

db.commit()

In [19]:
#del df['footnotes']
# creating column list for insertion
cols = "`,`".join([str(i) for i in extract_df.columns.tolist()])


cols


'Code'

In [ ]:
# Insert DataFrame recrds one by one.
for i,row in extract_df.iterrows():
    sql = "INSERT INTO `t_bls_Unemployed_codes` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    db.commit()

In [ ]:
df_final

import numpy as np
df_final = df_final.replace(np.nan, '', regex=True)

In [ ]:
# Prepare SQL query.
sql = "DROP TABLE t_fact_bls_Unemployed_data;"

try:
   # Execute the SQL command
    a = cursor.execute(sql)
    
   # Fetch all the rows in a list of lists.
   #rows = cursor.fetchall()
except:
    print ("Error: unable to fetch data");

db.commit()



In [38]:
print(df_final)

     year period periodName latest    value footnotes     seriesID
0    2020    M05        May   true  256.394            CUUR0000SA0
1    2020    M04      April         256.389            CUUR0000SA0
2    2020    M03      March         258.115            CUUR0000SA0
3    2020    M02   February         258.678            CUUR0000SA0
4    2020    M01    January         257.971            CUUR0000SA0
..    ...    ...        ...    ...      ...       ...          ...
123  2015    M06       June         237.657            CUSR0000SA0
124  2015    M05        May         237.001            CUSR0000SA0
125  2015    M04      April         236.222            CUSR0000SA0
126  2015    M03      March         235.976            CUSR0000SA0
127  2015    M02   February         235.342            CUSR0000SA0

[128 rows x 7 columns]


In [39]:
#del df['footnotes']
# creating column list for insertion
cols = "`,`".join([str(i) for i in df_final.columns.tolist()])


cols



'year`,`period`,`periodName`,`latest`,`value`,`footnotes`,`seriesID'

In [40]:
# Insert DataFrame recrds one by one.
for i,row in df_final.iterrows():
    sql = "INSERT INTO `t_fact_bls_Unemployed_data` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    db.commit()

In [18]:
sql = "select * from t_fact_bls_Unemployed_data;" 

try:
   # Execute the SQL command
   a = cursor.execute(sql)
    
   # Fetch all the rows in a list of lists.
   rows = cursor.fetchall()
except:
    print ("Error: unable to fetch data")

print("cursor.execute output: ",a)

df = "";

#Get columns names
try:
    hd = pd.DataFrame(i[0] for i in cursor.description)
    df = pd.DataFrame( [[ij for ij in i] for i in rows] )
    df.columns = hd[0]
    #df.head();
except:
    print ("Empty data returned.")

#Select head function. 
df

cursor.execute output:  5167


,stg_id,year,period,periodName,latest,value,footnotes,seriesID,ts
0,1,2020,M04,April,true,1816.0,,LNU03027659,2020-06-05 03:32:59
1,2,2020,M03,March,,745.0,,LNU03027659,2020-06-05 03:32:59
2,3,2020,M02,February,,671.0,,LNU03027659,2020-06-05 03:32:59
3,4,2020,M01,January,,711.0,,LNU03027659,2020-06-05 03:32:59
4,5,2019,M12,December,,588.0,,LNU03027659,2020-06-05 03:32:59
...,...,...,...,...,...,...,...,...,...
5162,5163,2005,M06,June,None,193.7,,CUSR0000SA0,2020-06-11 00:19:20
5163,5164,2005,M05,May,None,193.6,,CUSR0000SA0,2020-06-11 00:19:20
5164,5165,2005,M04,April,None,193.7,,CUSR0000SA0,2020-06-11 00:19:20
5165,5166,2005,M03,March,None,193.1,,CUSR0000SA0,2020-06-11 00:19:20


In [41]:
db.commit()
db.close()

Sites to checkout. 
https://www.dataquest.io/blog/sql-insert-tutorial/
https://datatofish.com/load-json-pandas-dataframe/
